# Case Study

Following the ETL, we're going to load each data source onto a data frame, clean it by checking for spelling mistakes, null values and errors, merge them all into one, and export the final master file.

In [1]:
# importing libraries
import pandas as pd
import json
import requests

### Sales Transactions

In [2]:
# opening the sales csv file and exploring it
sales = pd.read_csv('SalesTransactions.csv')

#### Data Cleaning: Null values and data types

In [3]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Source      1000 non-null   object 
 1   BranchId    1000 non-null   int64  
 2   CustomerId  1000 non-null   int64  
 3   UnItNr      1000 non-null   int64  
 4   CreateDate  1000 non-null   object 
 5   UpdateDate  329 non-null    object 
 6   UserId      1000 non-null   object 
 7   UserFied    1000 non-null   object 
 8   Amount      1000 non-null   float64
dtypes: float64(1), int64(3), object(5)
memory usage: 70.4+ KB



>**Observations:** No empty rows except for the `UpdateDate` column, which makes sense since not all transactions have to be updated. 
I also notice that the data type for the datetime columns is object instead of datetime. I will change that so we can sort rows by date.

In [4]:
#changing dtype to datetime
sales['CreateDate'] = pd.to_datetime(sales['CreateDate'])
sales['UpdateDate'] = pd.to_datetime(sales['UpdateDate'])

#### Data Transformation: UserFields column

From the information received from the IT department, I also know that the column `UserField` is entered manually, so it's going to have many different unique values that mean the same thing and should be unified. I'm going to investigate and resolve this issue now:

In [5]:
# correcting a typo
sales = sales.rename(columns={'UserFied': 'UserField'})

# checking for unique values
sales['UserField'].unique()

array(['3pm', 'Product: STR', 'Product: 3PC', '3PC', 'LTL',
       'Lont Term Lease', 'SHORT TERM RENTAL', 'ltl', 'STR',
       'Third   Party Contract', 'Third Party Contract',
       'ShortTermRental', 'LontTermLease', 'Product: LTL', 'str',
       'Short Term   Rental', 'Lont Term   Lease', 'lont term lease',
       'ThirdPartyContract', 'Short Term Rental               ',
       'Third Party Contract                     ',
       'THIRD PARTY CONTRACT', 'Short Term Rental', 'short term rental',
       'third party contract', 'LONT TERM LEASE',
       'Lont Term Lease                   '], dtype=object)

> **Observations:** I can see many different ways of indicating just a few values: long term lease, short term rental, and third party contract.

I need to create a data transformation step that will transform those values into their common form.

In [6]:
# defining the list of strings to transform
long_term_lease = ['LTL', 'Lont Term Lease', 'ltl', 'LontTermLease', 'Product: LTL',
                        'Lont Term   Lease', 'lont term lease', 'LONT TERM LEASE', 
                        'Lont Term Lease                   ']
short_term_rental = ['Product: STR', 'SHORT TERM RENTAL', 'STR', 'ShortTermRental',
                        'str', 'Short Term   Rental', 'Short Term Rental               ', 
                        'Short Term Rental','short term rental']
third_party_contract = ['3pm', 'Product: 3PC', '3PC', 'Third   Party Contract', 'Third Party Contract', 
                        'ThirdPartyContract', 'Third Party Contract                     ', 
                        'THIRD PARTY CONTRACT', 'third party contract']

# replacing the values
sales['UserField'] = sales['UserField'].replace(long_term_lease, 'long term lease')
sales['UserField'] = sales['UserField'].replace(short_term_rental, 'short term rental')
sales['UserField'] = sales['UserField'].replace(third_party_contract, 'third party contract')

# checking for unique values again
sales['UserField'].unique()

array(['third party contract', 'short term rental', 'long term lease'],
      dtype=object)

**Note:** I decided to go with the replace method because it best suits our goal of transforming this dataset into a report. However, if we were to need to make this is an automated pipeline, we could create a more complex transformation script that would identify which of the three categories it is, ignoring case, empty spaces, spelling mistakes, etc. Forexample, we could use functions such as `lower()`, `fuzzywuzzy`,  `textdistance`, `match_string()`, and others.

I also noticed the column `Source` had some repeated values with different spelling that should be unified.

In [7]:
sales['Source'].unique()

array(['ASSET APPLICATION', 'External Vendor', 'Ext. Vendor',
       'Asset Application', 'INV_5001', 'INV_5002', 'INV_5003',
       'INV_5004', 'INV_5005', 'INV_5006', 'INV_5007', 'INV_5008',
       'INV_5009', 'INV_5010', 'INV_5011', 'INV_5012', 'INV_5013',
       'INV_5014', 'INV_5015', 'INV_5016', 'INV_5017', 'INV_5018',
       'INV_5019', 'INV_5020', 'INV_5021', 'INV_5022', 'INV_5023',
       'INV_5024', 'INV_5025', 'INV_5026', 'INV_5027', 'INV_5028',
       'INV_5029', 'INV_5030', 'INV_5031', 'INV_5032', 'INV_5033',
       'INV_5034', 'INV_5035', 'INV_5036', 'INV_5037', 'INV_5038',
       'INV_5039', 'INV_5040', 'INV_5041', 'INV_5042', 'INV_5043',
       'INV_5044', 'INV_5045', 'INV_5046', 'INV_5047', 'INV_5048',
       'INV_5049', 'INV_5050', 'INV_5051', 'INV_5052', 'INV_5053',
       'INV_5054', 'INV_5055', 'INV_5056', 'INV_5057', 'INV_5058',
       'INV_5059', 'INV_5060', 'INV_5061', 'INV_5062', 'INV_5063',
       'INV_5064', 'INV_5065', 'INV_5066', 'INV_5067', 'INV_5068',
   

In [8]:
sales['Source'] = sales['Source'].replace('ASSET APPLICATION', 'Asset Application')
sales['Source'] = sales['Source'].replace('Ext. Vendor', 'External Vendor')

### Customers

In [9]:
# opening the sales csv file and exploring it
customers = pd.read_json(open('Customers.json'))

#### Data Cleaning: Null values and data types


In [10]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CustomerId      9 non-null      int64  
 1   CustomerName    9 non-null      object 
 2   CustomerParent  8 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 344.0+ bytes


In [11]:
customers

,CustomerId,CustomerName,CustomerParent
0,1,DHL,NaN
1,2,DHL-Post,1.0
2,3,DHL International GmbH,1.0
3,4,FedEx,4.0
4,5,fedex,4.0
5,6,FEDEX,4.0
6,7,Amazon,8.0
7,8,Amazon Logistics,7.0
8,9,Amazon Dilivery,8.0


>**Observations:** I notice that there seem there seem to be 3 main customers. It could be that DHL and DHL International are different clients, hence the need for the `CustomerParent` column. However I can also see several typos, which leads me to think it's manually entered and they are simply different names for the same customer. Additionally, the parent number for Amazon calls back to itself in an incoherent loop. But if there are only 3 customers I don't understand the need for the `CustomerParent` column, other than to converge the different spellings into one customer (except there's also a mistake here in Amazon's case).

I'm going to check how the sales table refers to the customer IDs to see if they're all in use.

In [12]:
sales['CustomerId'].unique()

array([6, 7, 4, 9, 8, 2, 1, 3, 5], dtype=int64)

Since they're all needed I'm only going to correct the typos and the `CustomerParent` issues, and change the data type of the `CustomerParent` to integer to better match the `CustomerId` column it's refering to.

In [13]:
# Correcting typo
customers['CustomerName'] = customers['CustomerName'].replace('Amazon Dilivery', 'Amazon Delivery')

# Filling in null value
customers.iloc[0, 2] = 1.0

# Correcting Parent issue
customers.iloc[7, 2] = 8.0

# Changing data type
customers['CustomerParent'] = customers['CustomerParent'].astype(int)
customers

,CustomerId,CustomerName,CustomerParent
0,1,DHL,1
1,2,DHL-Post,1
2,3,DHL International GmbH,1
3,4,FedEx,4
4,5,fedex,4
5,6,FEDEX,4
6,7,Amazon,8
7,8,Amazon Logistics,8
8,9,Amazon Delivery,8


Creating a hierarchy table of the customer data:

In [14]:
customer_hierarchy = customers.groupby(['CustomerParent']).agg({
                        'CustomerId': lambda x: list(set(x)), 
                        'CustomerName': lambda x: list(set(x))}).reset_index()

customer_hierarchy

,CustomerParent,CustomerId,CustomerName
0,1,"[1, 2, 3]","[DHL, DHL International GmbH , DHL-Post]"
1,4,"[4, 5, 6]","[FEDEX, fedex, FedEx]"
2,8,"[8, 9, 7]","[Amazon, Amazon Delivery, Amazon Logistics]"


### Employee API

In [15]:
# API config
url = "https://randomuser.me/api/"
params = {"seed": "SeedText"}

# Make the API call
response = requests.get(url, params=params)

# Convert the received data (dictionary) into a data frame
employee = pd.json_normalize(response.json()["results"])
employee.head(10)

,gender,email,phone,cell,nat,name.title,name.first,name.last,location.street.number,location.street.name,...,login.sha256,dob.date,dob.age,registered.date,registered.age,id.name,id.value,picture.large,picture.medium,picture.thumbnail
0,male,ron.ortiz@example.com,016974 34905,07391 543078,GB,Mr,Ron,Ortiz,1673,Stanley Road,...,ea6a72782433a5bbe242d39f4c855b260c60788c36450e...,2000-12-13T13:07:15.761Z,22,2012-02-21T19:42:58.906Z,11,NINO,TA 07 80 89 B,https://randomuser.me/api/portraits/men/23.jpg,https://randomuser.me/api/portraits/med/men/23...,https://randomuser.me/api/portraits/thumb/men/...


> **Observations:** I find it strange that there's only one row in this table, but this could be due to the fact that this is an excercise and not real data. 

I want to make sure that the request is being done properly, so I check that there's really one value by repeating the request several times:

In [16]:
# making multiple API calls and concatenating the results into a single DataFrame
dfs = []
for i in range(5):
    response = requests.get(url, params=params)
    data = response.json()["results"]
    df = pd.json_normalize(data)
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)

# view results
print(df)

  gender                  email         phone          cell nat name.title  \
0   male  ron.ortiz@example.com  016974 34905  07391 543078  GB         Mr   
1   male  ron.ortiz@example.com  016974 34905  07391 543078  GB         Mr   
2   male  ron.ortiz@example.com  016974 34905  07391 543078  GB         Mr   
3   male  ron.ortiz@example.com  016974 34905  07391 543078  GB         Mr   
4   male  ron.ortiz@example.com  016974 34905  07391 543078  GB         Mr   

  name.first name.last  location.street.number location.street.name  ...  \
0        Ron     Ortiz                    1673         Stanley Road  ...   
1        Ron     Ortiz                    1673         Stanley Road  ...   
2        Ron     Ortiz                    1673         Stanley Road  ...   
3        Ron     Ortiz                    1673         Stanley Road  ...   
4        Ron     Ortiz                    1673         Stanley Road  ...   

                                        login.sha256  \
0  ea6a72782433a5b

I'm going to check if the value of the `id.value` column matches any of the ids in our `sales` table

In [17]:
# checking as is
print(employee['id.value'].isin(sales['UserId']))

# checking without the spaces
employee_test = employee
employee_test['id.value'] = employee['id.value'].str.replace(' ', '')
print(employee_test['id.value'].isin(sales['UserId']))

0    False
Name: id.value, dtype: bool
0    False
Name: id.value, dtype: bool


>**Observation:** I assume there is no match because we're using this API as an excercise, so I will not merge this information with our table.

### Branches

For the sake of this excercise and with goal of obtaining a fully merged tablen and given how small the table is, I will manually create the Branch table based on the information provided by IT.

However, given this this table should be dynamic, this is not the best method. I will showcase a different method on a different platform.

In [18]:
branches = pd.DataFrame({
    'BranchId': [1, 2, 3, 4, 5, 6],
    'BranchName': ['Amsterdam', 'Rotterdam', 'Brugge', 'Berlin', 'Koln', 'Vienna'],
    'Country': ['Netherlands', 'Netherlands', 'Belgium', 'Germany', 'Germany', 'Austria'],
    'Region': ['Benelux', 'Benelux', 'Benelux','Central','Central','Central']
})

branches

,BranchId,BranchName,Country,Region
0,1,Amsterdam,Netherlands,Benelux
1,2,Rotterdam,Netherlands,Benelux
2,3,Brugge,Belgium,Benelux
3,4,Berlin,Germany,Central
4,5,Koln,Germany,Central
5,6,Vienna,Austria,Central


### Merging tables

In [19]:
# merging the sales and customers table
sales_customers = pd.merge(sales, customers, on='CustomerId')

# adding the branches table
sales_customers_branches = pd.merge(sales_customers, branches, on='BranchId')
sales_customers_branches.head()

,Source,BranchId,CustomerId,UnItNr,CreateDate,UpdateDate,UserId,UserField,Amount,CustomerName,CustomerParent,BranchName,Country,Region
0,Asset Application,5,6,1466,2021-03-09 10:50:00,2021-03-09 13:14:00,A87F7113,third party contract,443.23,FEDEX,4,Koln,Germany,Central
1,External Vendor,5,6,465,2021-03-09 11:46:00,NaT,FB167849,long term lease,261.14,FEDEX,4,Koln,Germany,Central
2,Asset Application,5,6,1014,2021-03-09 12:53:00,NaT,744D4C52,third party contract,344.66,FEDEX,4,Koln,Germany,Central
3,Asset Application,5,6,707,2021-03-09 13:27:00,NaT,C2BA5FC7,long term lease,280.05,FEDEX,4,Koln,Germany,Central
4,INV_5036,5,6,808,2021-03-09 15:37:00,2021-09-23 15:37:00,DCAF192F,long term lease,472.28,FEDEX,4,Koln,Germany,Central


### Export


In [22]:
# per product
group_product = sales_customers_branches.groupby('UnItNr')['Amount'].sum().reset_index()

# per location
group_location = sales_customers_branches.groupby('Country')['Amount'].sum().reset_index()

# per customer
group_customer = sales_customers_branches.groupby('CustomerId')['Amount'].sum().reset_index()

# per user
group_user = sales_customers_branches.groupby('UserId')['Amount'].sum().reset_index()

# export data to excel file in different sheets
writer = pd.ExcelWriter('result_data.xlsx')

sales_customers_branches.to_excel(writer, sheet_name='all', index=False)
group_location.to_excel(writer, sheet_name='location', index=False)
group_product.to_excel(writer, sheet_name='product', index=False)
group_customer.to_excel(writer, sheet_name='customer', index=False)
group_user.to_excel(writer, sheet_name='user', index=False)

writer.save()

Now we can access the data and open it as an excel file or on Power BI which I will showcase in the presentation.


### Conclusion

Throughout this project, a variety of data sources were utilized and processed using Python to clean and transform the data. The resulting data was then merged into a single table and loaded into Power BI to create a dashboard featuring clear and concise data visualizations. This process allowed for more efficient analysis and decision-making based on the provided data. While there is always room for improvement, this project highlights the benefits of using Python and Power BI to transform and analyze complex datasets.